In [ ]:
!pip install imutils

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
%matplotlib inline
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import os
import random
import cv2
import tensorflow as tf

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report,confusion_matrix
from sklearn import metrics
from sklearn.model_selection import train_test_split

from keras.models import Model,Sequential
from keras.layers import Dense,Flatten,Dropout,Conv2D,MaxPool2D,AveragePooling2D,Activation,Input
from keras.optimizers import Adam,SGD,RMSprop
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import img_to_array
from keras.applications import VGG16
from keras.preprocessing.image import ImageDataGenerator

from imutils import paths,build_montages

sns.set()

In [ ]:
imagePaths = sorted(list(paths.list_images('/kaggle/input/apparel-images-dataset')))

In [ ]:
random.seed(42)
random.shuffle(imagePaths)

In [ ]:
imagePaths[0]

In [ ]:
images = []
labels = []

for i in imagePaths:
    image = cv2.imread(i)
    image = cv2.resize(image , (96,96))
    image = img_to_array(image)
    images.append(image)
    
    label = i.split(os.path.sep)[-2]
    l = label.split('_')
    labels.append(l)


In [ ]:
images = np.array(images , dtype="float") / 255.0
labels = np.array(labels)

In [ ]:
labels

In [ ]:
images.shape

In [ ]:
mlb = MultiLabelBinarizer()

In [ ]:
labels = mlb.fit_transform(labels)

In [ ]:
labels.shape

In [ ]:
mlb.classes_

In [ ]:
(trainX , testX , trainY , testY) = train_test_split(images,labels,test_size=0.2,random_state=42)

In [ ]:
print(type(trainX))
print(trainX.shape)
print(type(trainY))
print(trainY.shape)
print(type(testX))
print(testX.shape)
print(type(testY))
print(testY.shape)

In [ ]:
aug = ImageDataGenerator(rotation_range=20,fill_mode='nearest',horizontal_flip=True,
                            shear_range=0.2,width_shift_range=0.1,height_shift_range=0.1,zoom_range=0.2)

In [ ]:
model = Sequential()

model.add(Conv2D(filters=16,kernel_size=(5,5),padding='same',input_shape=(96,96,3),activation="relu"))
model.add(MaxPool2D())
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Conv2D(filters=16,kernel_size=(3,3),padding='same',activation="relu"))
model.add(MaxPool2D())
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Conv2D(filters=32,kernel_size=(3,3),padding='same',activation="relu"))
model.add(MaxPool2D())
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(512, activation="relu"))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Dense(128, activation="relu"))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Dense(len(mlb.classes_), activation="sigmoid"))

In [ ]:
model.summary()

In [ ]:
INIT_LR = 1e-3
EPOCHS = 75
BS = 32

opt = SGD(learning_rate=INIT_LR)

In [ ]:
model.compile(loss="binary_crossentropy", optimizer=opt,metrics=["accuracy"])

In [ ]:
H = model.fit_generator(
	aug.flow(trainX, trainY, batch_size=BS),
	validation_data=(testX, testY),
	steps_per_epoch=len(trainX) // BS,epochs=EPOCHS)

In [ ]:
N = np.arange(0,EPOCHS)
plt.style.use('ggplot')
plt.figure(figsize=(8,6))
plt.plot(N , H.history['accuracy'], label='train_accuracy')
plt.plot(N , H.history['val_accuracy'], label='val_accuracy')
plt.plot(N , H.history['loss'], label='train_loss')
plt.plot(N , H.history['val_loss'], label='val_loss')
plt.legend()
plt.title('CNN for Multi-Label Classificartion')
plt.xlabel('#Epoch')
plt.ylabel('Loss-Accuracy Trade-off')
plt.show()

In [ ]:
Y_pred = model.predict(testX)
Y_pred

In [ ]:
Y_pred[0]

In [ ]:
testY[0]

In [ ]:
plt.imshow(testX[99])

In [ ]:
idxs = np.argsort(-1*Y_pred)
idxs

In [ ]:
idxs = idxs[:,:2]

In [ ]:
idxs

In [ ]:
plt.imshow(testX[0])

In [ ]:
mlb.classes_[idxs[0][0]]

In [ ]:
idxs[0][0]

In [ ]:
text1 = mlb.classes_[idxs[0][0]] + " : " + str(np.round(Y_pred[0][idxs[0][0]]*100,2))
text2 = mlb.classes_[idxs[0][1]] + " : " + str(np.round(Y_pred[0][idxs[0][1]]*100,2))
print(text1)
print(text2)

In [ ]:
text = text1 + " " + text2
text

In [ ]:
cv2.putText(testX[0], text, (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 1)
plt.imshow(testX[0])